# 2D Slice Generation from 3D NIfTI Files

The KiTS19 dataset includes 210 CT scans, each with expert manual segmentation annotations for kidney and tumor regions. These 3D NIfTI images provide a valuable resource for training and evaluating segmentation models, as they capture detailed organ and tumor structures across multiple cross-sectional slices. This notebook will guide the process of converting these 3D volumes into 2D slices to prepare data for multiclass semantic segmentation tasks.

### Objectives
1. **3D to 2D Slice Conversion**: Convert the 3D volumetric data into 2D slices for efficient training and computational simplicity in deep learning models.
2. **Multiclass Mask Generation**: Generate 2D binary masks with three distinct classes:
   - **Background**: Channel 1 (1, 0, 0)
   - **Kidney**: Channel 2 (0, 1, 0)
   - **Tumor**: Channel 3 (0, 0, 1)
   
This setup enables pixel-wise classification into background, kidney, and tumor classes, facilitating precise model training.

### Data Shape Specifications
- **Input Image**: Each slice will be represented in grayscale with a shape of `(512, 512, 1)`.
- **Mask**: The corresponding mask for each slice will be a three-channel binary image of shape `(512, 512, 3)`, where each channel indicates the presence of a specific class.

By running the next block of this notebook, within half an hour we will have generated a structured set of 2D slices from the original 3D CT volumes, enabling us to move forward with training a segmentation model that can accurately distinguish kidney and tumor regions.

In [ ]:
import os
import nibabel as nib
import numpy as np
import imageio
from PIL import Image

# Define the main paths
dataset_folder = r"C:\Users\pujau\OneDrive\Documents\3d\kits19\data"
output_folder = r"C:\Users\pujau\OneDrive\Documents\thesis\kits19\data\all_data"

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to normalize and convert the slices to 8-bit integers
def normalize_and_convert(slice_2d):
    # Normalize the slice to range [0, 255]
    slice_2d = 255 * (slice_2d - np.min(slice_2d)) / (np.max(slice_2d) - np.min(slice_2d))
    return slice_2d.astype(np.uint8)

# Function to save 2D slices
def save_slices(data, case_id, slice_type):
    case_output_folder = os.path.join(output_folder, case_id, slice_type)
    os.makedirs(case_output_folder, exist_ok=True)
    
    for i in range(data.shape[0]):
        slice_2d = data[i, :, :]
        slice_2d = normalize_and_convert(slice_2d)
        output_path = os.path.join(case_output_folder, f"{slice_type}_slice_{i:03d}.png")
        imageio.imwrite(output_path, slice_2d)

# Function to save 3-channel segmentation slices
def save_3_channel_slices(segmentation_data, case_id):
    case_output_folder = os.path.join(output_folder, case_id, "segmentation")
    os.makedirs(case_output_folder, exist_ok=True)
    
    num_classes = 3  # Define the number of classes (background, kidney, tumor)
    for i in range(segmentation_data.shape[0]):
        slice_2d = segmentation_data[i, :, :]
        
        # Create an empty 3-channel image
        multi_channel_slice = np.zeros((slice_2d.shape[0], slice_2d.shape[1], num_classes), dtype=np.uint8)
        
        # Assign each class to a specific channel
        for c in range(num_classes):
            class_slice = (slice_2d == c).astype(np.uint8)  
            multi_channel_slice[:, :, c] = class_slice
        
        output_path = os.path.join(case_output_folder, f"segmentation_slice_{i:03d}.png")
        imageio.imwrite(output_path, multi_channel_slice)

# Main loop over each case
for case_id in os.listdir(dataset_folder):
    case_folder = os.path.join(dataset_folder, case_id)
    
    if os.path.isdir(case_folder):
        imaging_path = os.path.join(case_folder, "imaging.nii.gz")
        segmentation_path = os.path.join(case_folder, "segmentation.nii.gz")
        
        if os.path.exists(imaging_path) and os.path.exists(segmentation_path):
            # Load the NIfTI files
            imaging = nib.load(imaging_path)
            segmentation = nib.load(segmentation_path)
            
            # Get the data arrays
            imaging_data = imaging.get_fdata()
            segmentation_data = segmentation.get_fdata()
            
            # Save 2D slices of imaging data
            save_slices(imaging_data, case_id, "imaging")
            
            # Save 3-channel segmentation slices
            save_3_channel_slices(segmentation_data, case_id)

